In [1]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)
config.__dict__

{'hyperparams': <util.tools.config at 0x279382b0400>,
 'programsettings': <util.tools.config at 0x279382b05e0>,
 'modelconfig': <util.tools.config at 0x279382b0880>}

In [4]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
all_data_source = []

for results_file in list_results_files:
#     print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
#     print("File Name:", results_file, ' \n results: ', lst_results)
    
    for results in lst_results:
        max_seq = 0
        lr = 0
        train_batch = 0
        max_seq = ""
        model_name = ""
        class_weights = '[]'
        kernel_1 = 0
        kernel_2 = 0
        kernel_3 = 0
        if (len(results[0]) > 50):
#             print(results[0])
            results_json = json.loads(results[0])
            max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
            lr = results_json["hyperparams"]["LEARNING_RATE"]
            train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
            model_name = results_json["programsettings"]["MODEL_NAME"]
            try:
                class_weights = results_json["hyperparams"]["LOSS_FN_CLASS_WEIGHTS"]
            
            except: 
                class_weights = '[]'
                
            try:
                kernel_1 = results_json["modelconfig"]["KERNEL_1"]
                kernel_2 = results_json["modelconfig"]["KERNEL_2"]
                kernel_3 = results_json["modelconfig"]["KERNEL_3"]
            
            except: 
                class_weights = '[]'

        result_store = [results_file, model_name,lr, train_batch,max_seq, class_weights,kernel_1, kernel_2,kernel_3,results[1], \
                        results[2], results[3], results[4], results[5],results[6], results[15], results[16] ]
        all_results.append(result_store)
        
#       Creating list of all dev_labels and dev_preds, so that we can run classification reports for all of them together
        if (len(results) > 7):
            all_data_source.append([results[7],results[8], results[13], results[14], result_store])
                                
    
columns = ['file_name','model_name', 'lr', 'train_batch_size','train_max_seq','class_weights','Kernel_1','Kernel_2','Kernel_3','train_loss', \
           'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score', 'test_mcc', 'test_f1_score' ]

df_results = pd.DataFrame(all_results, columns = columns)
pd.set_option('display.max_colwidth', 100)
df_results

,file_name,model_name,lr,train_batch_size,train_max_seq,class_weights,Kernel_1,Kernel_2,Kernel_3,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score,test_mcc,test_f1_score
0,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.024639,0.120250,0.985154,0.984075,0.960034,0.948919,0.956031,0.946364
1,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,256,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.012626,0.047154,0.993454,0.991550,0.982022,0.968613,0.978794,0.963900
2,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,384,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.007133,0.040432,0.996642,0.995055,0.985675,0.973941,0.983220,0.967283
3,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000008,12,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.025139,0.114040,0.985007,0.983913,0.959908,0.947789,0.955727,0.946643
4,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000013,12,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.023655,0.126698,0.985792,0.984727,0.958928,0.947031,0.955857,0.943452
5,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000015,12,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.023873,0.120178,0.985552,0.984802,0.960810,0.948111,0.957011,0.946163
6,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,16,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.024528,0.115474,0.985295,0.984038,0.960203,0.946053,0.955796,0.946487
7,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,24,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.025368,0.113962,0.985344,0.983732,0.960120,0.947203,0.957121,0.947241
8,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.024254,0.111488,0.985445,0.984262,0.962331,0.950136,0.957104,0.949163
9,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.024226,0.120995,0.985346,0.984152,0.959243,0.945536,0.957350,0.947572


### Let us generate Classification report for all the things we have available

In [5]:
for i in range(len(all_data_source)):
    dev_labels, dev_preds, test_preds, test_labels, model_config = all_data_source[i]
    print("\n \n \n",model_config)
    print(classification_report(dev_labels,dev_preds ))
    print(classification_report(test_labels,test_preds ))
    


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl'), 'BioBERT_fc', 1e-05, 12, 128, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 36, 48, 60, 0.024639492770546348, 0.12025048231940862, 0.9851540145895844, 0.9840753730218306, 0.960033703005189, 0.9489194175833915, 0.9560310245333161, 0.946363869595401]
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1034
           1       0.97      0.96      0.97      1145
           2       0.94      0.95      0.94      1382
           3       0.95      0.95      0.95      1269
           4       0.96      0.96      0.96       136
           5       0.96      0.92      0.94       954
           6       0.96      0.98      0.97       933
           7       0.91      0.85      0.88       250
           8       0.99      1.00      0.99      8201

    accuracy                           0.97     15304
   macro avg       0.95      0.94      0.95     15304
weig